In [ ]:
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import time
import yfinance as yf

In [ ]:
cik_store_priced = json.load(open("metastore/cik_store_priced.json", "r"))
company_tickers = pd.read_json("data/company_tickers.json")

In [ ]:
def ticker_to_cik(ticker):
    try:
        df = company_tickers.loc[ticker]
    except KeyError:
        return None
    
    if type(df) != pd.core.series.Series:
        return None
    else:
        return str(df.loc["cik_str"]).zfill(10)

In [ ]:
def clean_ticker(ticker):
    return ticker.split(" ")[0]
companies = pd.read_csv("spx.csv", sep=";")
companies.Ticker = companies.Ticker.apply(clean_ticker)
companies["cik"] = companies.Ticker.apply(ticker_to_cik)
companies.dropna(subset=["cik"], inplace=True)
companies = companies.sample(n=100, random_state=1337).set_index("cik").copy()

In [ ]:
sentiment_store = json.load(open("metastore/sentiment_store_item7.json", "r"))

In [ ]:
mean_sentiment_store = {}
for cik in tqdm(sentiment_store):
    mean_sentiment_store[cik] = {}
    cik_collection = sentiment_store[cik]
    for acc_nr in cik_collection:
        mean_sentiment = pd.DataFrame(cik_collection[acc_nr], columns=["positive", "negative", "neutral"]).mean()
        sentiment = mean_sentiment.iloc[0] - mean_sentiment.iloc[1]
        mean_sentiment_store[cik][acc_nr] = mean_sentiment.to_dict()
    json.dump(mean_sentiment_store, open("metastore/mean_sentiment_store_item7.json", "w"))